In [9]:
import pandas as pd

def is_consecutive(arr):
    # ex) [0,1,2,3] or [1,2,3,0], etc
    n = len(arr)
    for start in range(n):
        check = True
        for i in range(n):
            if arr[(start + i) % n] != i:
                check = False
                break
        if check:
            return True
    return False

def count_patterns(df):
    count_4con = 0   # 0123
    count_3con = 0   # 1230, 3012 등
    count_easy = 0   # 완전 역순, 2+2 split만
    easy_indices = []

    EASY_COMBINATIONS = [
        [3, 2, 1, 0],  # 완전 역순
        [2, 3, 0, 1],  # 2+2 split
        [1, 0, 3, 2],  # 2+2 split
    ]

    for i, row in df.iterrows():
        arr = [row[f"answer_{j}"] for j in range(4)]

        # Case 1: 4개 연속 (완전 정렬)
        if arr == [0, 1, 2, 3]:
            count_4con += 1
        # Case 2: 3개 연속 (예: 1230, 3012, 2310, 3120)
        elif is_consecutive(arr) and arr != [0, 1, 2, 3]:
            count_3con += 1
        # Case 3: "인식 쉬운 조합" (완전 역순, 2+2 split만)
        elif arr in EASY_COMBINATIONS:
            count_easy += 1
            easy_indices.append(i)
    return count_4con, count_3con, count_easy

In [11]:
a, b, c= count_patterns(df)
print("4개 연속:", a)
print("3개 연속:", b)
print("인식 쉬운 조합 개수:", c)

4개 연속: 304
3개 연속: 909
인식 쉬운 조합 개수: 618


In [12]:
from itertools import permutations
from collections import Counter

def count_all_permutations(df):
    # 0,1,2,3의 모든 순열을 생성 (4! = 24개)
    all_perms = list(permutations([0,1,2,3]))
    perm_counter = Counter()
    
    # 데이터에서 각 row의 answer_0~3을 튜플로 추출
    for _, row in df.iterrows():
        arr = tuple([row[f'answer_{i}'] for i in range(4)])
        perm_counter[arr] += 1

    # 24개 모든 순열의 등장 횟수 리스트(dict)
    result = {perm: perm_counter[perm] for perm in all_perms}
    return result


perm_counts = count_all_permutations(df)
for k, v in perm_counts.items():
    print(k, ":", v)


(0, 1, 2, 3) : 304
(0, 1, 3, 2) : 311
(0, 2, 1, 3) : 319
(0, 2, 3, 1) : 323
(0, 3, 1, 2) : 301
(0, 3, 2, 1) : 291
(1, 0, 2, 3) : 309
(1, 0, 3, 2) : 313
(1, 2, 0, 3) : 294
(1, 2, 3, 0) : 306
(1, 3, 0, 2) : 284
(1, 3, 2, 0) : 279
(2, 0, 1, 3) : 323
(2, 0, 3, 1) : 314
(2, 1, 0, 3) : 349
(2, 1, 3, 0) : 309
(2, 3, 0, 1) : 291
(2, 3, 1, 0) : 285
(3, 0, 1, 2) : 312
(3, 0, 2, 1) : 282
(3, 1, 0, 2) : 315
(3, 1, 2, 0) : 319
(3, 2, 0, 1) : 313
(3, 2, 1, 0) : 305


In [23]:
import pandas as pd
conclusion_keywords = [
    "마지막",
    "결론적",
    "끝",
    "요약",
    "정리",
    "종합",
    "한마디로",
    "전체적",
    "종합적",
    "따라서",
    "결국",
    "그러므로",
    "이상으로",
    "이와 같이",
    "앞서 언급한"
]

def find_conclusion_misplaced(df, keywords=conclusion_keywords):
    misplaced_rows = []
    for idx, row in df.iterrows():
        for i in range(4):
            sent = str(row[f"sentence_{i}"])
            ans = row[f"answer_{i}"]
            # 문장이 키워드로 시작하고, 순서가 3이 아니면
            if any(sent.startswith(keyword) for keyword in keywords):
                if ans != 3:
                    misplaced_rows.append({
                        "row_index": idx,
                        "sentence_idx": i,
                        "sentence": sent,
                        "answer": ans
                    })
    return pd.DataFrame(misplaced_rows)


result = find_conclusion_misplaced(df)
print(result)


      row_index  sentence_idx  \
0             4             2   
1             7             2   
2             8             2   
3            11             2   
4            22             2   
...         ...           ...   
1796       7337             2   
1797       7340             1   
1798       7342             0   
1799       7344             2   
1800       7348             0   

                                               sentence  answer  
0     결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...       0  
1                결국, 가계 소비 행태의 변화는 경제 성장에 중요한 변수로 작용한다.       1  
2                  결국, 도덕적 공동체의 진정한 힘은 포용성과 다양성에서 비롯된다.       1  
3         마지막으로, 두 요소의 조화는 전체적인 인테리어 스타일을 완성하는 데 필수적이다.       1  
4                결국, 지속 가능한 수질 관리를 통해 건강한 환경을 유지할 수 있다.       0  
...                                                 ...     ...  
1796  결국, 이러한 긴장은 사회적 관계와 개인의 정체성을 재구성하는 과정에서 중요한 역할...       1  
1797       결국, 미래 세대의 목소리를 반영하는 것은 모든 세대의 책임으로 여겨져야 한다.       0  
1798     

In [21]:
df.loc[11]

ID                                               TRAIN_0011
sentence_0                 벽지와 바닥재는 공간의 분위기를 결정짓는 중요한 요소이다.
sentence_1        바닥재는 내구성과 유지 관리의 용이성을 고려하여 결정하는 것이 바람직하다.
sentence_2    마지막으로, 두 요소의 조화는 전체적인 인테리어 스타일을 완성하는 데 필수적이다.
sentence_3             벽지의 색상과 패턴은 방의 크기와 조명을 고려하여 선택해야 한다.
answer_0                                                  0
answer_1                                                  3
answer_2                                                  1
answer_3                                                  2
Name: 11, dtype: object

In [13]:
df_au = pd.read_csv('train_augmented.csv')

In [14]:
perm_counts = count_all_permutations(df_au)
for k, v in perm_counts.items():
    print(k, ":", v)

(0, 1, 2, 3) : 304
(0, 1, 3, 2) : 311
(0, 2, 1, 3) : 319
(0, 2, 3, 1) : 323
(0, 3, 1, 2) : 301
(0, 3, 2, 1) : 291
(1, 0, 2, 3) : 309
(1, 0, 3, 2) : 7664
(1, 2, 0, 3) : 294
(1, 2, 3, 0) : 306
(1, 3, 0, 2) : 284
(1, 3, 2, 0) : 279
(2, 0, 1, 3) : 323
(2, 0, 3, 1) : 7665
(2, 1, 0, 3) : 349
(2, 1, 3, 0) : 309
(2, 3, 0, 1) : 291
(2, 3, 1, 0) : 285
(3, 0, 1, 2) : 7663
(3, 0, 2, 1) : 282
(3, 1, 0, 2) : 315
(3, 1, 2, 0) : 319
(3, 2, 0, 1) : 313
(3, 2, 1, 0) : 305


In [25]:
from sklearn.model_selection import train_test_split
import pandas as pd

def tuple_from_row(row):
    return tuple(row[f"answer_{i}"] for i in range(4))

# 각 row마다 순열 tuple 생성
df["permutation"] = df.apply(tuple_from_row, axis=1)

# stratify에 permutation 컬럼을 넣기!
train_df, val_df = train_test_split(
    df, 
    test_size=0.25, 
    random_state=42, 
    stratify=df["permutation"]
)

# 필요하다면 permutation 컬럼 삭제
train_df = train_df.drop(columns=["permutation"])
val_df = val_df.drop(columns=["permutation"])


In [26]:
perm_counts = count_all_permutations(train_df)
for k, v in perm_counts.items():
    print(k, ":", v)

(0, 1, 2, 3) : 228
(0, 1, 3, 2) : 233
(0, 2, 1, 3) : 239
(0, 2, 3, 1) : 242
(0, 3, 1, 2) : 226
(0, 3, 2, 1) : 218
(1, 0, 2, 3) : 232
(1, 0, 3, 2) : 235
(1, 2, 0, 3) : 221
(1, 2, 3, 0) : 229
(1, 3, 0, 2) : 213
(1, 3, 2, 0) : 209
(2, 0, 1, 3) : 242
(2, 0, 3, 1) : 235
(2, 1, 0, 3) : 262
(2, 1, 3, 0) : 232
(2, 3, 0, 1) : 218
(2, 3, 1, 0) : 214
(3, 0, 1, 2) : 234
(3, 0, 2, 1) : 212
(3, 1, 0, 2) : 236
(3, 1, 2, 0) : 239
(3, 2, 0, 1) : 235
(3, 2, 1, 0) : 229


In [27]:
perm_counts = count_all_permutations(val_df)
for k, v in perm_counts.items():
    print(k, ":", v)

(0, 1, 2, 3) : 76
(0, 1, 3, 2) : 78
(0, 2, 1, 3) : 80
(0, 2, 3, 1) : 81
(0, 3, 1, 2) : 75
(0, 3, 2, 1) : 73
(1, 0, 2, 3) : 77
(1, 0, 3, 2) : 78
(1, 2, 0, 3) : 73
(1, 2, 3, 0) : 77
(1, 3, 0, 2) : 71
(1, 3, 2, 0) : 70
(2, 0, 1, 3) : 81
(2, 0, 3, 1) : 79
(2, 1, 0, 3) : 87
(2, 1, 3, 0) : 77
(2, 3, 0, 1) : 73
(2, 3, 1, 0) : 71
(3, 0, 1, 2) : 78
(3, 0, 2, 1) : 70
(3, 1, 0, 2) : 79
(3, 1, 2, 0) : 80
(3, 2, 0, 1) : 78
(3, 2, 1, 0) : 76


In [28]:
df

,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3,permutation
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2,"(0, 3, 1, 2)"
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1,"(0, 3, 2, 1)"
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0,"(3, 2, 1, 0)"
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3,"(2, 0, 1, 3)"
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2,"(1, 3, 0, 2)"
...,...,...,...,...,...,...,...,...,...,...
7346,TRAIN_7346,인식론적 겸손은 자신의 지식의 한계를 인정하는 태도를 의미한다.,이러한 겸손은 새로운 정보와 관점을 수용하는 데 도움을 준다.,"지적 확신은 특정한 믿음이나 주장을 강하게 지지하는 태도로, 자신감 있는 결정을 가...","두 가지 태도의 균형은 비판적 사고를 촉진하고, 더 깊은 이해를 추구하는 데 필수적이다.",0,1,2,3,"(0, 1, 2, 3)"
7347,TRAIN_7347,거래 후에는 반드시 영수증이나 거래 내역을 기록하여 추후 문제 발생 시 증거로 활용...,중고 거래를 시작하기 전에 판매자와 구매자의 신뢰성을 확인하는 것이 중요하다.,거래 플랫폼의 리뷰와 평점을 참고하여 안전한 거래 상대를 선택할 수 있다.,"상품의 상태와 가격을 명확히 확인한 후, 직접 만나서 거래하는 것이 바람직하다.",1,2,3,0,"(1, 2, 3, 0)"
7348,TRAIN_7348,"결국, 소수자의 권리가 존중될 때 다수의 의견도 더욱 풍부해질 수 있다.","이 두 가지 원칙은 때때로 충돌할 수 있으며, 균형을 찾는 것이 필요하다.",소수자의 권리 보호는 민주주의의 핵심 요소로 여겨진다.,다수결 원칙은 사회의 다양한 의견을 반영하는 중요한 메커니즘이다.,2,3,1,0,"(2, 3, 1, 0)"
7349,TRAIN_7349,인간의 유전자를 수정하는 것이 자연의 섭리를 거스르는 행위로 여겨지기도 한다.,"결과적으로, 이러한 논의는 과학적 진보와 도덕적 책임 사이의 균형을 찾는 데 중요한...",유전자 편집 기술은 생명과학 분야에서 혁신적인 발전을 가져왔다.,그러나 이 기술의 윤리적 측면에 대한 논란이 끊이지 않고 있다.,2,3,0,1,"(2, 3, 0, 1)"


In [37]:
### 데이터 증강
import itertools
import random
from tqdm import tqdm

def hamming_distance(a, b):
    return sum(x != y for x, y in zip(a, b))

def augment_train_df(train_df, max_aug=2):
    """train_df에서 각 row별로 hard negative 증강(해밍거리 1~2), ID 변경"""
    augmented_rows = []
    for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
        orig_perm = tuple(row[f'answer_{i}'] for i in range(4))
        all_perms = list(itertools.permutations(range(4)))
        hard_perms = [p for p in all_perms if 1 <= hamming_distance(orig_perm, p) <= 2]
        if not hard_perms:
            continue
        selected = random.sample(hard_perms, min(max_aug, len(hard_perms)))
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        for idx, perm in enumerate(selected, 1):
            aug_sentences = [sentences[i] for i in perm]
            aug_row = {f'sentence_{i}': s for i, s in enumerate(aug_sentences)}
            for i in range(4):
                aug_row[f'answer_{i}'] = perm[i]
            # 증강 ID: 원본ID_1, _2, ...
            aug_row['ID'] = f"{row['ID']}_{idx}"
            augmented_rows.append(aug_row)
    aug_df = pd.DataFrame(augmented_rows)
    # 증강 데이터와 train_df 합치기 (원본은 ID 그대로, 증강은 _N)
    train_df_augmented = pd.concat([train_df, aug_df], ignore_index=True)
    return train_df_augmented




In [38]:
train_aug_df = augment_train_df(train_df, max_aug=2)

100%|██████████| 7351/7351 [00:00<00:00, 9732.78it/s] 


In [39]:
train_aug_df

,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2
...,...,...,...,...,...,...,...,...,...
22048,TRAIN_7348_2,소수자의 권리 보호는 민주주의의 핵심 요소로 여겨진다.,"결국, 소수자의 권리가 존중될 때 다수의 의견도 더욱 풍부해질 수 있다.","이 두 가지 원칙은 때때로 충돌할 수 있으며, 균형을 찾는 것이 필요하다.",다수결 원칙은 사회의 다양한 의견을 반영하는 중요한 메커니즘이다.,2,0,1,3
22049,TRAIN_7349_1,인간의 유전자를 수정하는 것이 자연의 섭리를 거스르는 행위로 여겨지기도 한다.,그러나 이 기술의 윤리적 측면에 대한 논란이 끊이지 않고 있다.,유전자 편집 기술은 생명과학 분야에서 혁신적인 발전을 가져왔다.,"결과적으로, 이러한 논의는 과학적 진보와 도덕적 책임 사이의 균형을 찾는 데 중요한...",0,3,2,1
22050,TRAIN_7349_2,유전자 편집 기술은 생명과학 분야에서 혁신적인 발전을 가져왔다.,"결과적으로, 이러한 논의는 과학적 진보와 도덕적 책임 사이의 균형을 찾는 데 중요한...",인간의 유전자를 수정하는 것이 자연의 섭리를 거스르는 행위로 여겨지기도 한다.,그러나 이 기술의 윤리적 측면에 대한 논란이 끊이지 않고 있다.,2,1,0,3
22051,TRAIN_7350_1,우주 관광 산업은 최근 몇 년간 급속히 발전하고 있다.,이러한 변화는 기술 혁신과 함께 대중의 관심을 끌고 있다.,민간 기업들이 우주 여행을 상업화하면서 새로운 시장이 열리고 있다.,"결과적으로, 우주 관광은 미래의 중요한 산업 중 하나로 자리잡을 가능성이 높다.",0,2,1,3


In [40]:
perm_counts = count_all_permutations(train_aug_df)
for k, v in perm_counts.items():
    print(k, ":", v)

(0, 1, 2, 3) : 942
(0, 1, 3, 2) : 908
(0, 2, 1, 3) : 951
(0, 2, 3, 1) : 893
(0, 3, 1, 2) : 909
(0, 3, 2, 1) : 888
(1, 0, 2, 3) : 922
(1, 0, 3, 2) : 912
(1, 2, 0, 3) : 942
(1, 2, 3, 0) : 922
(1, 3, 0, 2) : 888
(1, 3, 2, 0) : 913
(2, 0, 1, 3) : 940
(2, 0, 3, 1) : 931
(2, 1, 0, 3) : 914
(2, 1, 3, 0) : 923
(2, 3, 0, 1) : 921
(2, 3, 1, 0) : 908
(3, 0, 1, 2) : 920
(3, 0, 2, 1) : 893
(3, 1, 0, 2) : 948
(3, 1, 2, 0) : 900
(3, 2, 0, 1) : 938
(3, 2, 1, 0) : 927


In [ ]:
####

데이터 증강으로는 점수 증가 미미함